In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Step 1: Define Health Criteria

In [90]:
def to_snake_case(value):
    return "_".join(value.lower().split())

ingredient_map = {
    "Calories": "calories",
    "Total Fat": to_snake_case("Total Fat"),
    "Saturated Fat": to_snake_case("Saturated Fat"),
    "Trans Fat": to_snake_case("Trans Fat"),
    "Cholesterol": to_snake_case("Cholestorol"),
    "Sodium": "sodium",
    "Total Carbohydrate": "carbohydrates",
    "Dietary Fiber": "fiber",
    "Sugar": "sugar",
    "Added Sugars": to_snake_case("Added Sugars"),
    "Protein": "protein",
    "Vitamin A": "vitamin_a",
    "Calcium": "calcium",
    "Vitamin C": "vitamin_c",
    "Vitamin D": "vitamin_d",
    "Iron": "iron"
    
}


#Weights for overall health criteria (Subject to change)
health_criteria_weights = {
    'nutrient_density': 0.4,
    'macronutrient_balance': 0.3,
    'absence_of_harmful_ingredients': 0.2,
    'adherence_to_dietary_principles': 0.1
}

nutrient_density_criteria = {
    'vitamin_a' : {'threshold': 0.125, 'weight': 0.4},
    'vitamin_c' : {'threshold': 0.1, 'weight': 0.3},
    'vitamin_d' : {'threshold': 0.1, 'weight': 0.2},
    'fiber' : {'threshold': 3, 'weight': 0.2},
    'iron': {'threshold': 0.5, 'weight': 0.2}
}

macronutrient_balance_criteria = {
    'carbohydrates': {'threshold': 40, 'weight': 0.4},
    'protein': {'threshold': 20, 'weight': 0.3},
    'total_fat': {'threshold': 30, 'weight': 0.3}
}
    
absence_of_harmful_ingredients_criteria = {
    'trans_fat': {'threshold': 0.5, 'weight': 0.5},
    'sodium': {'threshold': 200, 'weight': 0.5},
    'added_sugars': {'threshold': 10, 'weight': 0.5}
}
    
# Criterion 4: Adherence to Dietary Principles
adherence_to_dietary_principles_criteria = {
    'gluten_free': {'value': True, 'weight': 1.0},
    'organic': {'value': True, 'weight': 0.5}
}

total_fat


# Step 2: Assign Weightings

In [91]:
nutrient_density_weight = health_criteria_weights['nutrient_density']
macronutrient_balance_weight = health_criteria_weights['macronutrient_balance']
absence_of_harmful_ingredients_weight = health_criteria_weights['absence_of_harmful_ingredients']
adherence_to_dietary_principles_weight = health_criteria_weights['adherence_to_dietary_principles']

# Step 3: Normalize Weights and Calculate Scores

In [106]:
def calculate_health_score(food_item):
    # Normalize Weights
    serving_size = food_item['serving_size']

    nutrient_density_score = calculate_weight_score(food_item, nutrient_density_criteria, 0)
    macronutrient_balance_score = calculate_weight_score(food_item, macronutrient_balance_criteria, 1)
    absence_of_harmful_ingredients_score = calculate_weight_score(food_item, absence_of_harmful_ingredients_criteria, 1)
    adherence_to_dietary_principles_score = calculate_weight_score(food_item, adherence_to_dietary_principles_criteria, 2)

    overall_score = (
        nutrient_density_weight * nutrient_density_score +
        macronutrient_balance_weight * macronutrient_balance_score +
        absence_of_harmful_ingredients_weight * absence_of_harmful_ingredients_score +
        adherence_to_dietary_principles_weight * adherence_to_dietary_principles_score
    )

    return overall_score
    


def calculate_weight_score(food_item, criteria, comparator):
    score = 0
    if comparator == 0:
        for nutrient, c in criteria.items():
            if nutrient in food_item and food_item[nutrient] >= c['threshold']:
                score += c['weight']
    elif comparator == 1:
        for nutrient, c in criteria.items():
            if nutrient in food_item and food_item[nutrient] <= c['threshold']:
                score += c['weight']
    else:
        for nutrient, c in criteria.items():
            if nutrient in food_item and food_item[nutrient] == c['value']:
                score += c['weight']

    return score


In [107]:
food_data = {
    'serving_size': 100,
    'calories': 250,
    'carbohydrates': 30,
    'protein': 15,
    'fat': 10,
    'added_sugars': 50,
    'vitamin_c': .5,
    'fiber': 4,
    'trans_fat': 0.2,
    'sodium': 180,
    'gluten_free': True,
    'organic': True
}

food_score = calculate_health_score(food_data)
print("Healthiness Score:", food_score)

Healthiness Score: 0.7200000000000001


# Step 4: Use Scanned Data to Calculate Score

In [108]:
filename = 'test_data/test1.txt'
with open(filename) as f:
    contents = [(line.strip()) for line in f.readlines()]

# print(contents)


food_dict = dict()
for i in range(1, len(contents)):
    curr = contents[i]
    split_str = contents[i].rsplit(" ", 1)
    # print(curr)
    print(split_str)
    if "Serving Size" in split_str[0]:
        food_dict["serving_size"] = int(''.join(filter(str.isdigit, split_str[1])))

    if "Percent" in split_str[0]:
        break

    elif split_str[0] in ingredient_map.keys() and len(split_str) == 1:
        next_split = contents[i+1].rsplit(" ", 1)
        food_dict[ingredient_map[split_str[0]]] = int(next_split[0])


        
    elif split_str[0] in ingredient_map.keys():
        if '%' in split_str[1]:
            split_str[1] = float(split_str[1].strip("%")) / 100
            food_dict[ingredient_map[split_str[0]]] = split_str[1]

        else: 
            food_dict[ingredient_map[split_str[0]]] = int(''.join(filter(str.isdigit, split_str[1])))
        

    # elif split_str[0] in ingredient_map.keys():
    #     food_dict[ingredient_map[split_str[0]]] = ''.join(filter(str.isdigit, split_str[1]))


food_dict

['Serving Size 3 oz.', '(85g)']
['Serving Per Container', '2']
['Amount Per', 'Serving']
['Calories']
['200']
['Calories from Fat', '120']
['Total Fat', '15g']
['Saturated Fat', '5g']
['Trans Fat', '3g']
['Cholesterol', '30mg']
['Sodium', '650mg']
['Total Carbohydrate', '30g']
['Dietary Fiber', '0g']
['Sugars', '5g']
['Protein', '5g']
['% Daily', 'Value*']
['20', '%']
['28%']
['10', '%']
['28', '%']
['10', '%']
['0%']
['Vitamin A', '5%']
['Calcium', '15%']
['Vitamin C', '2%']
['•']
['Iron', '5%']
['*Percent Daily Values are based on a 2,000 calorie', 'diet.']


{'serving_size': 85,
 'calories': 200,
 'total_fat': 15,
 'saturated_fat': 5,
 'trans_fat': 3,
 'cholestorol': 30,
 'sodium': 650,
 'carbohydrates': 30,
 'fiber': 0,
 'protein': 5,
 'vitamin_a': 0.05,
 'calcium': 0.15,
 'vitamin_c': 0.02,
 'iron': 0.05}

In [109]:
new_calculated_food_score = calculate_health_score(food_dict)
new_calculated_food_score

0.3